# Training and Testing A Visual Transformer Model
Here we test a visual Transfrom ViT from [An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale](https://arxiv.org/abs/2010.11929)

In [1]:
import torch
import torchvision as tv
import torchvision.transforms.v2 as v2
from our_datasets import Country_images
#from Country_dict import comp_country_dict
from ViT import VisionTransformer

USE_GPU = True
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
#get models
print(f"Using {device} device")

Using cuda device


## Load our Dataset and Create our dataloaders

In [2]:
batch_size = 64
weights = tv.models.DenseNet201_Weights.DEFAULT
transform = v2.Compose([weights.transforms(), ])


dataset_path = "data\\compressed_dataset\\"
dataset = Country_images("data\\compressed_dataset\\country_comp.csv",dataset_path,transform=transform)
num_classes = dataset.get_num_classes()
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset,lengths=[0.7,0.1,0.2])
data_loader_params = {
    'batch_size': batch_size,  # Batch size for data loading
    'num_workers': 10,  # Number of subprocesses to use for data loading
    'persistent_workers': True,  # If True, the data loader will not shutdown the worker processes after a dataset has been consumed once. This allows to maintain the worker dataset instances alive.
    'pin_memory': True,  # If True, the data loader will copy Tensors into CUDA pinned memory before returning them. Useful when using GPU.
    'pin_memory_device': 'cuda' ,  # Specifies the device where the data should be loaded. Commonly set to use the GPU.
}
train_dataloader      = torch.utils.data.DataLoader(train_dataset, **data_loader_params, shuffle=True)
val_dataloader        = torch.utils.data.DataLoader(val_dataset, **data_loader_params, shuffle=True)
test_dataloader       = torch.utils.data.DataLoader(test_dataset, **data_loader_params, shuffle=False,in_order=True)

## Load our model

In [3]:
#hyperparameters:
lr = 0.0001
#maximum learning rate we will let our model train in order to train faster at the start
max_lr = 0.01
weight_decay = 0.000001
EPOCHS = 100
#end hyperparameters

#model and optimizers
#model = tv.models.vit_b_16(weights=weights)
model = tv.models.densenet201(num_classes=num_classes)
#model = tv.models.vit_b_16(weights=weights)
#model = tv.models.DenseNet()
model.device = device
model.name = "DesnseNet-201"
model.path = "Trained_Models\\CNN\\" #where to save our best model
print(model.path)
#redfine our output layer to output our classes
#model.heads.head = torch.nn.Linear(in_features=model.heads.head.in_features,out_features=num_classes,bias=True)
print(model)
#loss_weights = torch.ones(num_classes,device=device)
#loss_weights[21] = 0.25
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
#scales the gradients, neccessary for mixed precision data types to properly converge
scaler = torch.amp.GradScaler(device=device)
#change our learning rate based on far we are in training and if we are improving
lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=max_lr, total_steps=EPOCHS*len(train_dataloader))

#added data to our model for ease of use (and to prevent passing so many variables to our training function)


Trained_Models\CNN\
DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats

## Train our Model

In [ ]:
#we will call the function we defined in "Training_Functions.py"
from Training_Functions import TrainModel
model_train = False

if model_train:
    model = model.to(device)
    TrainModel(model,EPOCHS, loss_fn, train_dataloader, val_dataloader, optimizer, lr_scheduler, scaler)
if not model_train:
    checkpoint = torch.load(model.path+model.name+"-best.pth")
    model.load_state_dict(checkpoint)
    model = model.to(device)

## Test our Model

In [5]:
#we will call the function we defined in "Training_Functions.py"
from Training_Functions import TestModel
TestModel(model, test_dataloader, loss_fn)

Got 6223 / 9598 correct (64.84)


0.6483642425505314